## Get Hyp3 data for the permafrost cal/val notebook

In [19]:
import numpy as np
import pandas as pd
from pathlib import Path
import asf_search as asf
from dateutil.parser import parse as parse_date
from typing import List, Union
from osgeo import gdal

In [2]:
# project_name = 'NC_BakersLake_Summer21'
# project_name = 'NC_NWT2_Summer19'
# project_name = 'NC_NWT3_Summer19' #location is further south
# project_name = 'NC_NWT3_Summer21'
# project_name = 'NC_Toolik_Summer20'
project_name = 'NC_Toolik_Summer21'
# project_name='NC_NSlopeWest_Summer21'
# project_name = 'NC_Kivalina_Summer19'
# project_name,year = 'Toolik',2023
project_name,year = 'Toolik',2023

path=None
frame=None

if project_name=='NC_BakersLake_Summer21': lonlat,year = 'POINT(-95.52900 64.839)',2021
if project_name=='NC_NWT2_Summer19': lonlat,year='POINT(-91.95 65.03)',2019
if project_name== 'NC_NWT3_Summer19': lonlat,year='POINT(-97.6848 61.9168)',2019
if project_name== 'NC_NWT3_Summer21': lonlat,year='POINT(-97.6848 61.9168)',2021
if project_name=='NC_Toolik_Summer20': lonlat,year='POINT(-149.37 69.09)',2020
if project_name=='NC_Toolik_Summer21': lonlat,year='POINT(-149.37 69.09)',2021
if project_name=='NC_NSlopeWest_Summer21': lonlat,year='POINT(-160.14 68.78)',2021
if project_name=='NC_Kivalina_Summer19': lonlat,year = 'POINT(-166.716 67.6078)',2019
if project_name=='Toolik': lonlat,path,frame='POINT(-149.37 69.09)',102,362
if project_name=='Toolik_alt': lonlat,frame='POINT(-149.37 69.09)',224
# if project_name=='ToolikNC':lonlat,path,frame='POINT(-149.37 69.09)',102,362

# work_dir = Path.home() / 'NISAR_cal_data' / project_name
work_dir = Path.home() / 'NISAR_cal_data' / project_name / str(year)
data_dir = work_dir / 'data'

stack_start = parse_date(f'{year}-06-01 00:00:00Z')
stack_end = parse_date(f'{year}-09-20 00:00:00Z')
max_temporal_baseline = 25 #days
if project_name == 'NC_NWT3_Summer21':
    max_temporal_baseline = 50 #days

data_dir.mkdir(parents=True, exist_ok=True)

# lonlat = 

In [3]:
# frame=None
direction = None
#note: using ASCENDING for this

# direction = asf.ASCENDING
# direction = asf.DESCENDING
if project_name=='NC_NSlopeWest_Summer21': frame,direction=362,asf.DESCENDING

search_results = asf.geo_search(
        platform=asf.SENTINEL1,
        intersectsWith=lonlat,
        start=f'{year}-05-25',
        end=f'{year}-09-10',
        processingLevel=asf.SLC,
        beamMode=asf.IW,
        flightDirection=direction,
        asfFrame=frame,
    )


In [4]:
baseline_results = asf.baseline_search.stack_from_product(search_results[-1])

columns = list(baseline_results[0].properties.keys()) + ['geometry', ]
data = [list(scene.properties.values()) + [scene.geometry, ] for scene in baseline_results]

stack = pd.DataFrame(data, columns=columns)
stack['startTime'] = stack.startTime.apply(parse_date)

stack = stack.loc[(stack_start <= stack.startTime) & (stack.startTime <= stack_end)]

print(len(stack))
stack

10


,beamModeType,browse,bytes,centerLat,centerLon,faradayRotation,fileID,flightDirection,groupID,granuleType,...,sceneName,sensor,startTime,stopTime,url,fileName,frameNumber,temporalBaseline,perpendicularBaseline,geometry
182,IW,None,4608855607,68.9321,-151.7568,None,S1A_IW_SLC__1SDV_20230603T164401_20230603T1644...,DESCENDING,S1A_IWDV_0361_0368_048824_102,SENTINEL_1A_FRAME,...,S1A_IW_SLC__1SDV_20230603T164401_20230603T1644...,C-SAR,2023-06-03 16:44:01+00:00,2023-06-03T16:44:29.000Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,S1A_IW_SLC__1SDV_20230603T164401_20230603T1644...,362,0,0.0,"{'coordinates': [[[-149.293701, 67.836151], [-..."
183,IW,None,4686505900,68.9316,-151.7565,None,S1A_IW_SLC__1SDV_20230615T164402_20230615T1644...,DESCENDING,S1A_IWDV_0361_0368_048999_102,SENTINEL_1A_FRAME,...,S1A_IW_SLC__1SDV_20230615T164402_20230615T1644...,C-SAR,2023-06-15 16:44:02.311000+00:00,2023-06-15T16:44:30.221Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,S1A_IW_SLC__1SDV_20230615T164402_20230615T1644...,362,12,-16.0,"{'coordinates': [[[-149.29332, 67.835739], [-1..."
184,IW,None,4688870133,68.9313,-151.7526,None,S1A_IW_SLC__1SDV_20230627T164402_20230627T1644...,DESCENDING,S1A_IWDV_0361_0368_049174_102,SENTINEL_1A_FRAME,...,S1A_IW_SLC__1SDV_20230627T164402_20230627T1644...,C-SAR,2023-06-27 16:44:02.446000+00:00,2023-06-27T16:44:30.361Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,S1A_IW_SLC__1SDV_20230627T164402_20230627T1644...,362,24,-172.0,"{'coordinates': [[[-149.289795, 67.835335], [-..."
185,IW,None,4678720995,68.9313,-151.7543,None,S1A_IW_SLC__1SDV_20230709T164403_20230709T1644...,DESCENDING,S1A_IWDV_0361_0368_049349_102,SENTINEL_1A_FRAME,...,S1A_IW_SLC__1SDV_20230709T164403_20230709T1644...,C-SAR,2023-07-09 16:44:03.895000+00:00,2023-07-09T16:44:31.808Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,S1A_IW_SLC__1SDV_20230709T164403_20230709T1644...,362,36,-107.0,"{'coordinates': [[[-149.291443, 67.835373], [-..."
186,IW,None,4689162092,68.9314,-151.7562,None,S1A_IW_SLC__1SDV_20230721T164404_20230721T1644...,DESCENDING,S1A_IWDV_0361_0368_049524_102,SENTINEL_1A_FRAME,...,S1A_IW_SLC__1SDV_20230721T164404_20230721T1644...,C-SAR,2023-07-21 16:44:04.486000+00:00,2023-07-21T16:44:32.398Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,S1A_IW_SLC__1SDV_20230721T164404_20230721T1644...,362,48,-23.0,"{'coordinates': [[[-149.293121, 67.835464], [-..."
187,IW,None,4678115605,68.9315,-151.7550,None,S1A_IW_SLC__1SDV_20230802T164404_20230802T1644...,DESCENDING,S1A_IWDV_0361_0368_049699_102,SENTINEL_1A_FRAME,...,S1A_IW_SLC__1SDV_20230802T164404_20230802T1644...,C-SAR,2023-08-02 16:44:04.784000+00:00,2023-08-02T16:44:32.697Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,S1A_IW_SLC__1SDV_20230802T164404_20230802T1644...,362,60,-107.0,"{'coordinates': [[[-149.29213, 67.83564], [-14..."
188,IW,None,4614840459,68.9313,-151.7536,None,S1A_IW_SLC__1SDV_20230814T164405_20230814T1644...,DESCENDING,S1A_IWDV_0361_0368_049874_102,SENTINEL_1A_FRAME,...,S1A_IW_SLC__1SDV_20230814T164405_20230814T1644...,C-SAR,2023-08-14 16:44:05.666000+00:00,2023-08-14T16:44:33.581Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,S1A_IW_SLC__1SDV_20230814T164405_20230814T1644...,362,72,-110.0,"{'coordinates': [[[-149.290665, 67.835297], [-..."
189,IW,None,4671684100,68.9310,-151.7538,None,S1A_IW_SLC__1SDV_20230826T164406_20230826T1644...,DESCENDING,S1A_IWDV_0361_0368_050049_102,SENTINEL_1A_FRAME,...,S1A_IW_SLC__1SDV_20230826T164406_20230826T1644...,C-SAR,2023-08-26 16:44:06.579000+00:00,2023-08-26T16:44:34.495Z,https://datapool.asf.alaska.edu/SLC/SA/S1A_IW_...,S1A_IW_SLC__1SDV_20230826T164406_20230826T1644...,362,84,-118.0,"{'coordinates': [[[-149.291, 67.83493], [-148...."
190,IW,None,4628564907,68.9322,-151.7518,None,S1A_IW_SLC__1SDV_20230907T164407_20230907T1644...,DESCENDING,S1A_IWDV_0361_0368_050224_102,SENTINEL_1A_FRAME,...,S1A_IW_SLC__1SDV_20230907T164407_20230907T1644...,C-SAR,2023-09-07 16:44:07.029000+00:00,2023-09-07T16:44:34.946Z,https://d

In [5]:
sbas_pairs = set()

for reference, rt in stack.loc[::-1, ['sceneName', 'temporalBaseline']].itertuples(index=False):
    secondaries = stack.loc[
        (stack.sceneName != reference)
        & (stack.temporalBaseline - rt <= max_temporal_baseline)
        & (stack.temporalBaseline - rt > 0)
    ]
    for secondary in secondaries.sceneName:
        sbas_pairs.add((reference, secondary))

## Do Hyp3 processing

In [6]:

import hyp3_sdk as sdk

hyp3 = sdk.HyP3(prompt=True)


NASA Earthdata Login username:  acjohnson16
NASA Earthdata Login password:  ········


In [7]:

queuejobs = False
jobs = sdk.Batch()
h3projname = f"{project_name}_NCV_{year}"
if queuejobs:
    for reference, secondary in sbas_pairs:
        jobs += hyp3.submit_insar_job(reference, secondary, name=h3projname,
                                      include_dem=True, include_look_vectors=True)

if queuejobs: print('Jobs were set to be processed by this command!')
else: print('No jobs queued. If you wish to process commands set queuejobs to True')


No jobs queued. If you wish to process commands set queuejobs to True


In [11]:
#watch jobs. this could be slow to run

# jobs = hyp3.watch(jobs)


In [8]:
class Error(Exception):
    """Base class for other exceptions"""
    pass
class stillrunningError(Error):
    """raised when jobs are still running"""
    pass

running = len(hyp3.find_jobs(status_code=["RUNNING"]))
running += len(hyp3.find_jobs(status_code=["PENDING"]))

if running > 0:
    print('Jobs still running!')
    raise stillrunningError
else:
    print('No jobs running, all clear')

No jobs running, all clear


In [11]:
h3projname = f"{project_name}_NCV_{year}"
jobs = hyp3.find_jobs(name=h3projname)

In [12]:
#now get the data

insar_products = jobs.download_files(data_dir)
insar_products = [sdk.util.extract_zipped_product(ii) for ii in insar_products]


  0%|          | 0/17 [00:00<?, ?it/s]

S1AA_20230802T164404_20230826T164406_VVP024_INT80_G_ueF_7DCC.zip:   0%|          | 0/226319665 [00:00<?, ?it/s…

S1AA_20230907T164407_20230919T164407_VVP012_INT80_G_ueF_0EC4.zip:   0%|          | 0/222879464 [00:00<?, ?it/s…

S1AA_20230721T164404_20230802T164404_VVP012_INT80_G_ueF_359F.zip:   0%|          | 0/223574203 [00:00<?, ?it/s…

S1AA_20230627T164402_20230709T164403_VVP012_INT80_G_ueF_93ED.zip:   0%|          | 0/224482924 [00:00<?, ?it/s…

...

S1AA_20230615T164402_20230627T164402_VVP012_INT80_G_ueF_9CDF.zip:   0%|          | 0/222324959 [00:00<?, ?it/s…

S1AA_20230627T164402_20230721T164404_VVP024_INT80_G_ueF_3070.zip:   0%|          | 0/225341304 [00:00<?, ?it/s…

S1AA_20230615T164402_20230709T164403_VVP024_INT80_G_ueF_2D1F.zip:   0%|          | 0/223802169 [00:00<?, ?it/s…

S1AA_20230826T164406_20230907T164407_VVP012_INT80_G_ueF_E23A.zip:   0%|          | 0/221125422 [00:00<?, ?it/s…

S1AA_20230802T164404_20230814T164405_VVP012_INT80_G_ueF_2787.zip:   0%|          | 0/224075419 [00:00<?, ?it/s…

In [17]:
#clip to common extent

def get_common_overlap(file_list: List[Union[str, Path]]) -> List[float]:
    """Get the common overlap of  a list of GeoTIFF files
    
    Arg:
        file_list: a list of GeoTIFF files
    
    Returns:
         [ulx, uly, lrx, lry], the upper-left x, upper-left y, lower-right x, and lower-right y
         corner coordinates of the common overlap
    """
    
    corners = [gdal.Info(str(dem), format='json')['cornerCoordinates'] for dem in file_list]

    ulx = max(corner['upperLeft'][0] for corner in corners)
    uly = min(corner['upperLeft'][1] for corner in corners)
    lrx = min(corner['lowerRight'][0] for corner in corners)
    lry = max(corner['lowerRight'][1] for corner in corners)
    return [ulx, uly, lrx, lry]

def clip_hyp3_products_to_common_overlap(data_dir: Union[str, Path], overlap: List[float]) -> None:
    """Clip all GeoTIFF files to their common overlap
    
    Args:
        data_dir:
            directory containing the GeoTIFF files to clip
        overlap:
            a list of the upper-left x, upper-left y, lower-right-x, and lower-tight y
            corner coordinates of the common overlap
    Returns: None
    """

    
    files_for_mintpy = ['_water_mask.tif', '_corr.tif', '_unw_phase.tif', '_dem.tif', '_lv_theta.tif', '_lv_phi.tif']

    for extension in files_for_mintpy:

        for file in data_dir.rglob(f'*{extension}'):

            dst_file = file.parent / f'{file.stem}_clipped{file.suffix}'

            gdal.Translate(destName=str(dst_file), srcDS=str(file), projWin=overlap)
            


In [20]:
files = data_dir.glob('*/*_dem.tif')
overlap = get_common_overlap(files)

clip_hyp3_products_to_common_overlap(data_dir, overlap)